In [2]:
import pandas as pd
import numpy as np
import sqlite3
from bokeh.io import output_notebook, save
from bokeh.plotting import show
from bokeh.layouts import gridplot
from bokeh.models import FactorRange

import CleanData as cd
import BokehPlots as bp
import CoinagePlots as cp

# import sys
# stdout = sys.stdout
# reload(sys)
# sys.setdefaultencoding('utf-8')
# sys.stdout = stdout

In [3]:
title = 'Roman_Imperial_Coinage'
fname = title+'.sqlite'
conn = sqlite3.connect(fname)

## British Museum

In [7]:
source = 'britishMuseum'
query = '''
WITH merged AS (
  SELECT
    emperor,
    denomination,
    description,
    start,
    end
  FROM
    {} JOIN emperors ON
      startDate >= emperors.start
      AND {}.startDate <= emperors.end
)
SELECT
  emperor,
  denomination,
  description,
  start,
  end
FROM merged
WHERE description LIKE '%radiate%'
ORDER BY start, end, denomination;
'''.format(source, source)
bm = pd.read_sql(query, conn)

DatabaseError: Execution failed on sql '
WITH merged AS (
  SELECT
    emperor,
    denomination,
    description,
    start,
    end
  FROM
    britishMuseum JOIN emperors ON
      startDate >= emperors.start
      AND britishMuseum.startDate <= emperors.end
)
SELECT
  emperor,
  denomination,
  description,
  start,
  end
FROM merged
WHERE description LIKE '%radiate%'
ORDER BY start, end, denomination;
': no such table: britishMuseum

# OLD STYLE (BAD)

In [19]:
museum = 'britishMuseum'
mints_bm = cd.readQuery(fname, museum)
mints_bm['dates'] = mints_bm['dates'].apply(cd.stringToList(valType=int))

In [23]:
emperors = np.load('../Data/Emperors.npy').item()

In [24]:
def findEmperor(df, emperors, authorityCol='authority', datesCol='dates'):
    '''
    '''
    df['emperor'] = ''
    for emperor, dates in emperors.iteritems():
        df['emperor'][df[authorityCol].str.contains(emperor)] = emperor
        matchDates = cp.coinsFromDates(df, dates, col_name='dates')
        df['emperor'][matchDates] = emperor
    return df

In [25]:
emperors

{'Aemilianus': (253, 253),
 'Anthemius': (467, 472),
 'Antoninus Pius': (138, 161),
 'Arcadius': (383, 408),
 'Augustus': (-27, 14),
 'Aurelian': (270, 275),
 'Avitus': (455, 456),
 'Balbinus': (238, 238),
 'Caligula': (37, 41),
 'Caracalla': (198, 217),
 'Carinus': (283, 285),
 'Carus': (282, 283),
 'Claudius': (41, 54),
 'Claudius Gothicus': (268, 270),
 'Commodus': (177, 192),
 'Constans I': (337, 350),
 'Constantine II': (337, 340),
 'Constantine III': (409, 411),
 'Constantine the Great': (306, 337),
 'Constantius I': (305, 306),
 'Constantius II': (337, 361),
 'Constantius III': (421, 421),
 'Decius': (249, 251),
 'Didius Julianus': (193, 193),
 'Diocletian': (284, 305),
 'Domitian': (81, 96),
 'Elagabalus': (218, 222),
 'Florianus': (276, 276),
 'Galba': (68, 69),
 'Galerius': (305, 311),
 'Gallienus': (253, 268),
 'Geta': (209, 211),
 'Glycerius': (473, 474),
 'Gordian I': (238, 238),
 'Gordian II': (238, 238),
 'Gordian III': (238, 244),
 'Gratian': (367, 383),
 'Hadrian': (11

In [6]:
mints_with_desc = mints_bm[mints_bm['description'] != '']
mints_with_desc = findEmperor(mints_bm, emperors)

/Users/William/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/William/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
plot_title = cp.makeTitle(subjects=['radiate'])
good_df = cp.containKeyword(mints_with_desc, ['radiate'], ['description'])
good_dfs = dict()
orderedEmperors = [x[0] for x in sorted(emperors.items(), key=lambda x: x[1][0])]

if not good_df.empty:
    good_dfs[plot_title] = good_df
    bar_plot = cp.makeCoinageStackedBar(good_df, x_col='emperor', stack_col='denomination', 
                                        title=plot_title+' Stacked Bar Graph', sort_bars=False,
                                        sort_x=False, sort_stacks=False, stack_order=orderedEmperors)

show(bar_plot)